# Store Sales - Time Series Forecasting

## IMPORT

In [36]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

In [2]:
#import database
df_holidays_events = pd.read_csv('holidays_events.csv')
df_oil = pd.read_csv('oil.csv')
df_sample_submission = pd.read_csv('sample_submission.csv')
df_stores = pd.read_csv('stores.csv')
df_test = pd.read_csv('test.csv')
df_train = pd.read_csv('train.csv')
df_transactions = pd.read_csv('transactions.csv')

In [3]:
#print out the shape of all tables
print("Shape of holidays_events: " + str(df_holidays_events.shape))
print("Shape of oil: " + str(df_oil.shape))
print("Shape of sample_submission: " + str(df_sample_submission.shape))
print("Shape of stores: " + str(df_stores.shape))
print("Shape of test: " + str(df_test.shape))
print("Shape of train: " + str(df_train.shape))
print("Shape of transactions: " + str(df_transactions.shape))

Shape of holidays_events: (350, 6)
Shape of oil: (1218, 2)
Shape of sample_submission: (28512, 2)
Shape of stores: (54, 5)
Shape of test: (28512, 5)
Shape of train: (3000888, 6)
Shape of transactions: (83488, 3)


In [4]:
df_train.head(5)

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [5]:
df_test.head(5)

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


## Now we will merge the tables

In [6]:
#merge train dataset and stores, use store_nbr column as key column
df_stores.fillna(method='bfill',inplace=True)
df_train = pd.merge(df_train, df_stores, on="store_nbr", how="left")
df_test = pd.merge(df_test, df_stores, on="store_nbr", how="left")
df_train

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,Quito,Pichincha,D,13
1,1,2013-01-01,1,BABY CARE,0.000,0,Quito,Pichincha,D,13
2,2,2013-01-01,1,BEAUTY,0.000,0,Quito,Pichincha,D,13
3,3,2013-01-01,1,BEVERAGES,0.000,0,Quito,Pichincha,D,13
4,4,2013-01-01,1,BOOKS,0.000,0,Quito,Pichincha,D,13
...,...,...,...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha,B,6
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha,B,6
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha,B,6
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha,B,6


In [7]:
#merge train dataset and oil, use date column as key column
df_oil.fillna(method='bfill',inplace=True)
df_train = pd.merge(df_train, df_oil, on="date", how="left")
df_test = pd.merge(df_test, df_oil, on="date", how="left")
df_train

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,Quito,Pichincha,D,13,93.14
1,1,2013-01-01,1,BABY CARE,0.000,0,Quito,Pichincha,D,13,93.14
2,2,2013-01-01,1,BEAUTY,0.000,0,Quito,Pichincha,D,13,93.14
3,3,2013-01-01,1,BEVERAGES,0.000,0,Quito,Pichincha,D,13,93.14
4,4,2013-01-01,1,BOOKS,0.000,0,Quito,Pichincha,D,13,93.14
...,...,...,...,...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha,B,6,47.57
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha,B,6,47.57
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha,B,6,47.57
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha,B,6,47.57


In [8]:
#merge train dataset and transactions, use date & store_nbr column as key column
df_transactions.fillna(method='bfill',inplace=True)
df_train = pd.merge(df_train, df_transactions, on=["date","store_nbr"], how="left")
df_test = pd.merge(df_test, df_transactions, on=["date","store_nbr"], how="left")
df_train

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,transactions
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,Quito,Pichincha,D,13,93.14,NaN
1,1,2013-01-01,1,BABY CARE,0.000,0,Quito,Pichincha,D,13,93.14,NaN
2,2,2013-01-01,1,BEAUTY,0.000,0,Quito,Pichincha,D,13,93.14,NaN
3,3,2013-01-01,1,BEVERAGES,0.000,0,Quito,Pichincha,D,13,93.14,NaN
4,4,2013-01-01,1,BOOKS,0.000,0,Quito,Pichincha,D,13,93.14,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha,B,6,47.57,2155.0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha,B,6,47.57,2155.0
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha,B,6,47.57,2155.0
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha,B,6,47.57,2155.0


In [9]:
#merge train dataset and holidays, use date column as key column
df_holidays_events.fillna(method='bfill',inplace=True)
df_train = pd.merge(df_train, df_holidays_events, on="date", how="left")
df_test = pd.merge(df_test, df_holidays_events, on="date", how="left")
df_train

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,dcoilwtico,transactions,type_y,locale,locale_name,description,transferred
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,Quito,Pichincha,D,13,93.14,NaN,Holiday,National,Ecuador,Primer dia del ano,False
1,1,2013-01-01,1,BABY CARE,0.000,0,Quito,Pichincha,D,13,93.14,NaN,Holiday,National,Ecuador,Primer dia del ano,False
2,2,2013-01-01,1,BEAUTY,0.000,0,Quito,Pichincha,D,13,93.14,NaN,Holiday,National,Ecuador,Primer dia del ano,False
3,3,2013-01-01,1,BEVERAGES,0.000,0,Quito,Pichincha,D,13,93.14,NaN,Holiday,National,Ecuador,Primer dia del ano,False
4,4,2013-01-01,1,BOOKS,0.000,0,Quito,Pichincha,D,13,93.14,NaN,Holiday,National,Ecuador,Primer dia del ano,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,3000883,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha,B,6,47.57,2155.0,Holiday,Local,Riobamba,Fundacion de Riobamba,False
3054344,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha,B,6,47.57,2155.0,Holiday,Local,Riobamba,Fundacion de Riobamba,False
3054345,3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha,B,6,47.57,2155.0,Holiday,Local,Riobamba,Fundacion de Riobamba,False
3054346,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha,B,6,47.57,2155.0,Holiday,Local,Riobamba,Fundacion de Riobamba,False


## Analyze data

In [10]:
# converting date to datetime
df_train.date = pd.to_datetime(df_train.date)
df_test.date = pd.to_datetime(df_test.date)


In [11]:
by_state = df_train.groupby(["state"]).sum().sort_values(by="sales", ascending=True)
by_state.index = by_state.index.astype(str)
by_state

,id,store_nbr,sales,onpromotion,cluster,dcoilwtico,transactions
state,,,,,,,
Pastaza,85061571144,1244364,4.201150e+06,102040,395934,2.737169e+06,1.706578e+07
Chimborazo,85044772230,791868,1.161044e+07,115743,395934,2.737169e+06,7.686347e+07
Santa Elena,85067170782,1414050,1.165464e+07,116200,56562,2.737169e+06,5.108278e+07
Imbabura,85046638776,848430,1.166691e+07,123008,848430,2.737169e+06,7.425416e+07
Bolivar,85054104960,1074678,1.331201e+07,114292,848430,2.737169e+06,7.083318e+07
Esmeraldas,85104501702,2432166,1.672204e+07,127227,565620,2.737169e+06,7.332382e+07
Loja,85093302426,2149356,1.924817e+07,150331,226248,2.737169e+06,9.647748e+07
Cotopaxi,170083944822,1414050,2.160778e+07,226844,1696860,5.474338e+06,1.188083e+08
Manabi,255375101124,8993358,2.563804e+07,396155,1527174,8.211506e+06,9.817668e+07


In [32]:
# custom template for plotly
custom_template_go = {
    "layout": go.Layout(
        font={
            "size": 12,
            "color": "#666",
        },
        title={
            "font": {
                "family": "Times New Roman",
                "size": 18,
                "color": "#666",
            },
        },
        plot_bgcolor="#ffffff",
        paper_bgcolor="#ffffff",
        xaxis={
            "showgrid": False,
            "zeroline": False,
        },
        yaxis={
            "showgrid": False,
            "zeroline": False,
        },
        margin=dict(b=20,r=60,l=70,t=115),
    )
}

In [33]:
fig = go.Figure(go.Bar(y=by_state.index, x=by_state.sales, 
                       orientation="h", marker_color="limegreen",
                       text=by_state.sales, textposition="outside", texttemplate='%{text:.2s}'))
fig.update_layout(template=custom_template_go,
                    margin=dict(b=20,r=60,l=80,t=115),
                    title={'text': "<span style='font-size:45px; font-family:Times New Roman'>Sales by State ($)</span><br><br>",
                    'y':0.85, 'x':0.5,
                    'xanchor': 'center', 'yanchor': 'top'},
                    )
fig.show()

In [37]:
by_city = df_train[["city", "state", "sales", "store_nbr"]].groupby(["city"]).agg({'sales':'sum', 'state':'first', 'store_nbr':'nunique'}).sort_values(by="sales", ascending=False) # grouping sales by city and state

fig = px.bar(y=by_city.index, x=by_city.sales, orientation="h", color=by_city.state, 
             text=by_city.sales, 
             color_discrete_sequence=["#000000", "#145214", "#d6f5d6", "#2eb82e", "#85e085", "#00ff00", "#669900", "#b3ff1a", "#c6ff1a", "#00ff99", "#00b3b3", "#006666", "#003333", "#669999", "#ffbf00", "#663300"],
             labels={"color": "State"})

# updating text position and formatting
fig.update_traces(texttemplate='%{text:.2s}', textposition="outside")
# add text annotation
fig.add_annotation(x=by_city.sales.max()/2.5, y=by_city.index[-1], text="The state with most cities has only 4,<br>and not all cities <br>in their respective state has the same sales pattern", showarrow=False, font=dict(color="black", size=12), xanchor="left", yanchor="top")
# updating figure layout
fig.update_layout(template=custom_template_go,
                    plot_bgcolor='#fafafa', paper_bgcolor='#fafafa',
                    margin=dict(b=20,r=60,l=70,t=115),
                    title={'text': '<span style="font-size:45px; color=#444; font-family:Times New Roman">Sales by City ($)</span><br>grouped by each state<br><br>',
                            'y':0.85, 'x':0.5,
                            'xanchor': 'center', 'yanchor': 'top'}, 
                    )
fig.show()

In [39]:
by_family = df_train[["family", "sales"]].groupby("family").sum().sort_values(by="sales")
by_family.index = by_family.index.str.title() # capitalizing family names

fig = go.Figure(go.Bar(y=by_family.index[-10:], x=by_family.sales[-10:], 
                       orientation="h", marker_color="limegreen",
                       text=by_family.sales[-10:], textposition="outside", texttemplate='%{text:.2s}'))
fig.update_layout(template=custom_template_go,
                    margin=dict(b=20,r=40,l=200,t=115),
                    title={'text': "<span style='font-size:45px; font-family:Times New Roman'>Top 10 Product Categories ($)</span><br><br>",
                    'y':0.85, 'x':0.5,
                    'xanchor': 'center', 'yanchor': 'top'},
                    xaxis_title="Sales",
                    yaxis_title="Product Categories",
                    )
fig.show()

In [40]:
other = by_family[:-10].sum()
top10 = by_family[-10:].sum()

fig = go.Figure()
fig.add_trace(go.Pie(labels=["Top 10 Categories", "Others"], values=[top10.sales, other.sales], marker_colors=["2eb82e", "#d6f5d6"],))
fig.update_layout(template=custom_template_go,
                  title={'text': "<span style='font-size:45px; font-family:Times New Roman'>Top 10 Categories vs. Rest</span><br><span style='color:#2eb82e'>Top 10</span> represents <span style='color:#2eb82e'>93%</span> of the sales<br><br>",
                    'y':0.85, 'x':0.5,
                    'xanchor': 'center', 'yanchor': 'top'},
)
fig.show()

In [59]:
cm = by_city[["sales", "store_nbr"]].sort_values(by="store_nbr", ascending=False)
cm

,sales,store_nbr
city,,
Quito,5.686793e+08,18
Guayaquil,1.255722e+08,8
Cuenca,5.019405e+07,3
Santo Domingo,3.661757e+07,3
Manta,1.432667e+07,2
Ambato,4.115977e+07,2
Machala,3.409467e+07,2
Latacunga,2.160778e+07,2
Playas,7.822669e+06,1


## Quito is the city with the most sales and also has the largest number of stores compared to other cities.

In [61]:
fig = go.Figure(go.Bar(y=cm.index, x=cm.store_nbr, 
                       orientation="h", marker_color="limegreen"))
fig.update_layout(template=custom_template_go,
                    margin=dict(b=20,r=40,l=200,t=115),
                    xaxis_title="Sales",
                    yaxis_title="Product Categories",
                    )
fig.show()